In [34]:
import anthropic
import json
import pandas as pd
import textwrap
import os
import re
from dotenv import load_dotenv

pd.set_option('display.float_format', lambda x: '%.3f' % x)
idx = pd.IndexSlice

load_dotenv()

client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [2]:
ANIMALS = ['Cattle', 'Chickens', 'Ducks', 'Goats', 'Pigs',
           'Sheep', 'Carp', 'Other Fish', 'Salmon', 'Shrimp']

In [5]:
df = pd.read_csv('data/Animal Welfare Grants - All.csv')
df["Amount"] = df["Amount"].replace("[$,]", "", regex=True).astype(float)
df.to_csv('data/grants.csv')

print(df.head().to_string())

   Year       Organization                                     Recipient      Amount                                                                   Description
0  2024  Open Philanthropy                            Rethink Priorities 3253550.000              Rethink Priorities — Farm Animal Welfare Research Support (2023)
1  2024  Open Philanthropy                 Plant Based Foods Association 2500000.000             Plant Based Foods Association — Advocacy and Corporate Engagement
2  2024  Open Philanthropy  European Institute for Animal Law and Policy 1819991.000  European Institute for Animal Law & Policy — EU Farm Animal Welfare Advocacy
3  2024  Open Philanthropy                   Compassion in World Farming 1504198.000                            Compassion in World Farming — Farmed Fish Welfare 
4  2024  Open Philanthropy                            Focus Philanthropy  880000.000                   Focus Philanthropy — Farm Animal Welfare Fundraising (2024)


In [46]:
def classify(row):
    prompt = str(textwrap.dedent("""Recipient: "{{recipient}}"
Description: "{{description}}"
Step 1: Analyze the grant to determine the type of animal.

Look for keywords related to "Farmed" (e.g., livestock, poultry, aquaculture), "Wild" (e.g., wildlife, conservation), or "Companion" (e.g., pets, shelters) animals.
If no clear indication of animal type, mark as "Unknown".
Analysis: {animal_type_analysis}
Animal Type: {animal_type}

Step 2: Identify the specific animal mentioned in the grant, if any.

Search for mentions of "Cattle", "Chickens", "Ducks", "Goats", "Pigs", "Sheep", "Carp, "Salmon", or "Shrimp".
If the grant benefits fish but does not specify a type, mark as "Other Fish".
If no specific animal is mentioned or if the animal doesn't match the listed categories, mark as "Not Specified".
Analysis: {specific_animal_analysis}
Specific Animal: {specific_animal}

Step 3: Determine the specific country mentioned in the grant, if any.

Look for mentions of country names or regions in the recipient or description.
If no specific country is mentioned, mark as "Not Specified".
Avoid false positives by ensuring the country name is in the proper context.
Analysis: {country_analysis}
Country: {country}

Final Classification:
                                 
{
    "animal_type": "{animal_type}",
    "specific_animal": "{specific_animal}",
    "country": "{country}"
}
    """)).replace('{{recipient}}', str(row['Recipient'])).replace('{{description}}', str(row['Description']))
    # print(prompt)
    try:
        response = client.messages.create(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1024,
            temperature=0.0,
            model="claude-3-haiku-20240307")
        result = response.content[0].text.strip()
        json_match = re.search(r'({.*?})\s*$', result, re.DOTALL)
        if json_match:
            classification_json = json_match.group(1)
            print("Final Classification:", classification_json)
            return json.loads(classification_json)
        else:
            print("No JSON object found in the response.")
        return {}
        # return pd.Series(json.loads(result))
    except Exception as e:
        print(e)
        return {}
        # return pd.Series({'Animal': 'Unknown', 'Country': 'Unknown'})

In [47]:
row = df.iloc[15]
# print(row)
result = classify(row)

Final Classification: {
    "animal_type": "Farmed",
    "specific_animal": "Not Specified",
    "country": "Not Specified"
}


In [48]:
result

{'animal_type': 'Farmed',
 'specific_animal': 'Not Specified',
 'country': 'Not Specified'}

In [49]:
df_test = df.iloc[:10]

In [50]:
c = df.apply(lambda x: classify(x), axis=1)

Final Classification: {
    "animal_type": "Farmed",
    "specific_animal": "Not Specified",
    "country": "Not Specified"
}
Final Classification: {
    "animal_type": "Unknown",
    "specific_animal": "Not Specified",
    "country": "Not Specified"
}
Final Classification: {
    "animal_type": "Farmed",
    "specific_animal": "Not Specified",
    "country": "European Union"
}
Final Classification: {
    "animal_type": "Aquaculture",
    "specific_animal": "Other Fish",
    "country": "Not Specified"
}
Final Classification: {
    "animal_type": "Farmed",
    "specific_animal": "Not Specified",
    "country": "Not Specified"
}
Final Classification: {
    "animal_type": "Wild",
    "specific_animal": "Other Fish",
    "country": "Netherlands"
}
Final Classification: {
    "animal_type": "Farmed",
    "specific_animal": "Not Specified",
    "country": "Not Specified"
}
Final Classification: {
    "animal_type": "Farmed",
    "specific_animal": "Not Specified",
    "country": "United State

In [18]:
df[c.columns] = c

In [6]:
df

,Year,Organization,Recipient,Amount,Description
0,2024,Open Philanthropy,Rethink Priorities,3253550.0,Rethink Priorities — Farm Animal Welfare Resea...
1,2024,Open Philanthropy,Plant Based Foods Association,2500000.0,Plant Based Foods Association — Advocacy and C...
2,2024,Open Philanthropy,European Institute for Animal Law and Policy,1819991.0,European Institute for Animal Law & Policy — E...
3,2024,Open Philanthropy,Compassion in World Farming,1504198.0,Compassion in World Farming — Farmed Fish Welf...
4,2024,Open Philanthropy,Focus Philanthropy,880000.0,Focus Philanthropy — Farm Animal Welfare Fundr...
...,...,...,...,...,...
1062,2016,Open Philanthropy,Humane Society of the United States,500000.0,Humane Society of the United States — Corporat...
1063,2016,Open Philanthropy,The Pollination Project,233000.0,The Pollination Project — Global Animal Advoca...
1064,2016,Open Philanthropy,Fórum Nacional de Proteção e Defesa Animal,100000.0,Fórum Nacional de Proteção e Defesa Animal — I...
1065,2016,Open Philanthropy,People for Animals,89392.0,People for Animals — International Cage-Free A...


In [22]:
pt = df.pivot_table(index='Country', columns='Animal', values='Amount', aggfunc='sum', fill_value=0)
print(pt.to_string())

Animal                                 Carp       Cattle  Cattle, Chickens, Ducks, Goats, Pigs, Sheep  Cattle, Chickens, Goats, Pigs, Sheep     Chickens  Chickens, Carp, Other Fish    Ducks   Other Fish       Pigs  Pigs, Chickens  Rats/Mice   Salmon     Shrimp       Unknown
Country                                                                                                                                                                                                                                                                           
Africa                                  0.0         0.00                                          0.0                                   0.0    125510.00                         0.0      0.0     39900.00        0.0             0.0        0.0      0.0        0.0  2.610000e+05
Africa, Asia, Latin America             0.0         0.00                                          0.0                                   0.0         0.00                       

In [27]:
s = pt.sum()
print(s)

Animal
Carp                                             269000.000
Cattle                                         56910997.150
Cattle, Chickens, Ducks, Goats, Pigs, Sheep      100000.000
Cattle, Chickens, Goats, Pigs, Sheep             462974.000
Chickens                                       67406347.110
Chickens, Carp, Other Fish                       175000.000
Ducks                                             84000.000
Other Fish                                     37015640.150
Pigs                                            5462581.000
Pigs, Chickens                                    40000.000
Rats/Mice                                          7500.000
Salmon                                           202457.000
Shrimp                                          4976966.000
Unknown                                       194967893.340
dtype: float64


In [33]:
(s['Carp'] + s['Other Fish'] + s['Salmon'])/(-1951699833.697 + -1897440352.025 +  -35457490.468)

-0.00965018781218219

In [34]:
s['Chickens']/-4208769312.000

-0.01601569060053059

In [36]:
s['Shrimp']/-2018785541.017

-0.0024653267515938136

In [35]:
s['Pigs']/-202723782.386

-0.026945930742348082

In [37]:
s['Cattle']/-156494452.036

-0.36366143597798717